## Read the data

In [15]:
import pandas as pd
import numpy as np
import json

In [16]:
monthly_data = pd.read_csv('monthly_surgery.csv', encoding='EUC-KR')

# lets drop row which does not have more than "-" in the "Surgery" column
monthly_data = monthly_data[monthly_data['Surgery'].str.count("-") >= 2]

# Add "Reference Surgery" column to the dataframe with the str split
monthly_data['Reference Surgery'] = monthly_data['Surgery'].str.split("-").str[1]

In [17]:
# get Monthly data where column "Monthly" is "Total"
monthly_data_total = monthly_data[monthly_data['Monthly'] == 'Total']

In [18]:
january_data = monthly_data[monthly_data["Monthly"] == "January"]
february_data = monthly_data[monthly_data["Monthly"] == "February"]
march_data = monthly_data[monthly_data["Monthly"] == "March"]
april_data = monthly_data[monthly_data["Monthly"] == "April"]
may_data = monthly_data[monthly_data["Monthly"] == "May"]
june_data = monthly_data[monthly_data["Monthly"] == "June"]
july_data = monthly_data[monthly_data["Monthly"] == "July"]
august_data = monthly_data[monthly_data["Monthly"] == "August"]
september_data = monthly_data[monthly_data["Monthly"] == "September"]
october_data = monthly_data[monthly_data["Monthly"] == "October"]
november_data = monthly_data[monthly_data["Monthly"] == "November"]
december_data = monthly_data[monthly_data["Monthly"] == "December"]

# check count of data
print("Total data count: ", len(monthly_data_total))
print("January data count: ", len(january_data))
print("February data count: ", len(february_data))
print("March data count: ", len(march_data))
print("April data count: ", len(april_data))
print("May data count: ", len(may_data))
print("June data count: ", len(june_data))
print("July data count: ", len(july_data))
print("August data count: ", len(august_data))
print("September data count: ", len(september_data))
print("October data count: ", len(october_data))
print("November data count: ", len(november_data))
print("December data count: ", len(december_data))


Total data count:  198
January data count:  198
February data count:  198
March data count:  198
April data count:  198
May data count:  198
June data count:  198
July data count:  198
August data count:  198
September data count:  198
October data count:  198
November data count:  198
December data count:  198


In [19]:
blood_demand_per_patient = {}

operation_blood_data = pd.read_csv('operation_blood_demand.csv')

for index, row in operation_blood_data.iterrows():
    if row["Transfused patients No."] != 0:
        blood_demand_per_patient[row["Operation"].lower()] = row["Total units transfused"] / row["Transfused patients No."]

In [20]:
blood_demand_per_patient

# save as json
with open('blood_demand_per_patient.json', 'w') as f:
    json.dump(blood_demand_per_patient, f)

In [21]:
monthly_data_total["Reference Surgery"]

12      Bilateral thyroidectomy 
13      Bilateral thyroidectomy 
14      Bilateral thyroidectomy 
15      Bilateral thyroidectomy 
16      Bilateral thyroidectomy 
                 ...            
211       Liver transplantation 
212       Liver transplantation 
213       Liver transplantation 
214       Liver transplantation 
215       Liver transplantation 
Name: Reference Surgery, Length: 198, dtype: object

Check if everything is allright

In [22]:
set(monthly_data_total["Reference Surgery"].values) - set(february_data["Reference Surgery"].values)

set()

In [23]:
data_dict = {}
# data_dict["Total"] = None
for month in monthly_data["Monthly"].unique():
    data_dict[month] = None

In [24]:
year_columns = monthly_data.columns[4:-2]
year_columns

monthly_dict = {}

for year in year_columns:
    for month in monthly_data["Monthly"].unique():
        if month == "Total":
            continue
        monthly_dict[year + "_" + month] = None

monthly_dict

{'2006_January': None,
 '2006_February': None,
 '2006_March': None,
 '2006_April': None,
 '2006_May': None,
 '2006_June': None,
 '2006_July': None,
 '2006_August': None,
 '2006_September': None,
 '2006_October': None,
 '2006_November': None,
 '2006_December': None,
 '2007_January': None,
 '2007_February': None,
 '2007_March': None,
 '2007_April': None,
 '2007_May': None,
 '2007_June': None,
 '2007_July': None,
 '2007_August': None,
 '2007_September': None,
 '2007_October': None,
 '2007_November': None,
 '2007_December': None,
 '2008_January': None,
 '2008_February': None,
 '2008_March': None,
 '2008_April': None,
 '2008_May': None,
 '2008_June': None,
 '2008_July': None,
 '2008_August': None,
 '2008_September': None,
 '2008_October': None,
 '2008_November': None,
 '2008_December': None,
 '2009_January': None,
 '2009_February': None,
 '2009_March': None,
 '2009_April': None,
 '2009_May': None,
 '2009_June': None,
 '2009_July': None,
 '2009_August': None,
 '2009_September': None,
 '2009_

In [25]:
monthly_data[monthly_data["Monthly"] == "Total"]["2013"]

12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
       ..
211   NaN
212   NaN
213   NaN
214   NaN
215   NaN
Name: 2013, Length: 198, dtype: float64

In [28]:
for key in monthly_dict.keys():
    year = key.split("_")[0]
    month = key.split("_")[1]
    rows = monthly_data[(monthly_data["Monthly"] == month)]
    # print(rows)
    # find rows that have "Surgery Received Person[person]" in "Category"
    person_rows = rows[rows["Category"].str.contains("Surgery Received Person")]
    for index, row in person_rows.iterrows():
        reference_surgery = row["Reference Surgery"]
        blood_usage_per_patient = blood_demand_per_patient[str(reference_surgery).lower().strip()]
        # if year == "2013" and month == "Total":
        #     print(reference_surgery, blood_usage_per_patient, row[year])
        #     print(np.isnan(row[year]))
            
        if not np.isnan(row[year]):
            value = blood_usage_per_patient * row[year]
            if np.isnan(value):
                print(blood_demand_per_patient, row[year])
            if monthly_dict[key] is None:
                monthly_dict[key] = blood_usage_per_patient * row[year]
            else:
                monthly_dict[key] += blood_usage_per_patient * row[year]



In [29]:
print(monthly_dict)

# save as json
with open('monthly_blood_demand.json', 'w') as f:
    json.dump(monthly_dict, f)

# create yearly dict

yearly_dict = {}
for key in monthly_dict:
    year = key.split("_")[0]
    if year not in yearly_dict:
        yearly_dict[year] = 0
    yearly_dict[year] += monthly_dict[key]

with open('yearly_blood_demand.json', 'w') as f:
    json.dump(yearly_dict, f)

{'2006_January': 191110.07616420163, '2006_February': 180227.7323187022, '2006_March': 180959.53747129586, '2006_April': 166051.4777870185, '2006_May': 176506.98369339452, '2006_June': 173967.56674422562, '2006_July': 181515.71795819214, '2006_August': 194730.8743609112, '2006_September': 172933.82271531963, '2006_October': 167959.73937949925, '2006_November': 172427.67755778495, '2006_December': 172807.27417203685, '2007_January': 226536.2641982038, '2007_February': 183491.7034105756, '2007_March': 202043.26373188524, '2007_April': 187988.69276279546, '2007_May': 194828.91492518422, '2007_June': 185737.1974823699, '2007_July': 202971.55482596645, '2007_August': 210325.0498153015, '2007_September': 176626.7553836934, '2007_October': 197818.7401217883, '2007_November': 182565.42259547368, '2007_December': 177587.6832773818, '2008_January': 229778.2324474649, '2008_February': 180804.11057800057, '2008_March': 196318.37737733603, '2008_April': 183388.91352748653, '2008_May': 183326.446798

#### JSON common items. Year, Month, Source, 

Now we need to calculate the city demand

In [30]:
# preprocess the monthly dict to be fit for Morgan's data structure

morgans_data = []

for key in monthly_dict:
    year = key.split("_")[0]
    month = key.split("_")[1]
    morgans_data.append({
        "year": year,
        "month": month,
        "value": monthly_dict[key]
    })

# save as json
with open('morgans_data.json', 'w') as f:
    json.dump(morgans_data, f)